In [1]:
from dash import Dash, html, dcc, callback, Output, Input, dash_table
import plotly.express as px
import pandas as pd, numpy as np
pd.set_option('display.max_columns', None)
import sqlite3, sqlalchemy 

In [2]:
def generate_seasons(start, stop):
    years = [y for y in range(start, stop)] 
    seasons = [str(y) + '_' + str(y+1)[2:] for y in years]  
    return seasons 
print(generate_seasons(2007, 2024))

['2007_08', '2008_09', '2009_10', '2010_11', '2011_12', '2012_13', '2013_14', '2014_15', '2015_16', '2016_17', '2017_18', '2018_19', '2019_20', '2020_21', '2021_22', '2022_23', '2023_24']


In [3]:
engine = sqlalchemy.create_engine('sqlite:///../../DB/ballbase.db') 
seasons = generate_seasons(2006, 2024)
dfs = [] 

for i in range(len(seasons)):
    df = pd.read_sql('master_'+seasons[i], con=engine) 
    df['Season'] = seasons[i].replace('_', '-')
    dfs.append(df) 

def convert_numeric(column): 
    return pd.to_numeric(column.astype(str).str.replace('%', ''))

master = (pd.concat(dfs, axis=0)
          .rename(columns={'TRB':'RPG', 'AST':'APG', 'PTS':'PPG'}) 
)

master[['PG%','SG%', 'SF%', 'PF%', 'C%']] = (master[['PG%','SG%', 'SF%', 'PF%', 'C%']]
                                            .fillna(0)
                                            .apply(convert_numeric)
)

master['3P%']

0        NaN
1        0.0
2      0.337
3        NaN
4        0.0
       ...  
567    0.143
568    0.373
569    0.208
570    0.333
571      NaN
Name: 3P%, Length: 8989, dtype: object

In [4]:


temp = master.groupby('Season').apply(lambda x: np.round(x.mean(numeric_only=True, skipna=True), 3)).reset_index() 
temp['Player'] = 'League Average'
temp['Pos'] = 'All' 
temp['Tm'] = 'All' 
master['3P'] / master['3PA'] 

C:\Users\bchan\AppData\Local\Temp\ipykernel_18124\4084981635.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp = master.groupby('Season').apply(lambda x: np.round(x.mean(numeric_only=True, skipna=True), 3)).reset_index()


ZeroDivisionError: float division by zero

In [8]:
master[master['Season']=='2009-10']

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,RPG,APG,STL,BLK,TOV,PF,PPG,3PAr,AST%,BLK%,BPM,DBPM,DRB%,DWS,FTr,OBPM,ORB%,OWS,PER,STL%,TOV%,TRB%,TS%,USG%,VORP,WS,WS/48,DRtg,ORtg,And1,BadPass,Blkd,C%,LostBall,Off.,Off..1,On-Off,OnCourt,PF%,PG%,PGA,SF%,SG%,Shoot,Shoot.1,T2P,T2PA,T3PA,TAST,TBLK,TDRB,TFG,TFGA,TFT,TFTA,TMP,TORB,TPF,TPTS,TSTL,TTOV,TTRB,Season
0,Arron Afflalo,SG,24,DEN,82,75,27.1,3.3,7.1,0.465,1.3,3.0,0.434,2.0,4.1,0.488,0.557,0.9,1.2,0.735,0.7,2.4,3.1,1.7,0.6,0.4,0.9,2.7,8.8,0.426,9.3,1.0,-0.4,-0.2,9.9,1.4,0.168,-0.2,3.1,2.8,10.9,1.0,10.5,6.5,0.576,14.0,0.9,4.3,0.092,111.0,114.0,12.0,20.0,50.0,0,19.0,11.0,9.0,-1.5,3.6,0,0,297.0,7,92,121.0,47.0,666,1317,351,231,84,518,794,1668,756,840,3239,105,171,2472,112,271,623,2009-10
1,Alexis Ajinça,C,21,CHA,6,0,5.0,0.8,1.7,0.500,0.0,0.0,NaN,0.8,1.7,0.500,0.500,0.0,0.2,0.000,0.2,0.5,0.7,0.0,0.2,0.2,0.3,0.8,1.7,0.000,0.0,2.7,-5.3,1.0,11.8,0.0,0.100,-6.3,4.1,-0.1,6.3,1.8,16.1,8.0,0.479,19.3,0.0,0.0,-0.013,103.0,81.0,1.0,0.0,1.0,100,0.0,1.0,2.0,9.1,10.6,0,0,0.0,0,0,2.0,1.0,639,1141,387,651,77,483,768,1528,593,773,2966,71,119,2258,125,261,554,2009-10
2,LaMarcus Aldridge,PF,24,POR,78,78,37.5,7.4,15.0,0.495,0.1,0.2,0.313,7.4,14.8,0.498,0.497,2.9,3.9,0.757,2.5,5.6,8.0,2.1,0.9,0.6,1.3,3.0,17.9,0.014,9.9,1.3,1.2,-0.2,18.6,3.3,0.260,1.4,8.1,5.5,18.2,1.3,7.4,13.3,0.535,22.9,2.3,8.8,0.145,107.0,113.0,38.0,31.0,42.0,12,27.0,30.0,12.0,3.6,4.9,88,0,370.0,0,0,129.0,144.0,646,1268,243,501,82,266,719,1511,534,702,2792,107,181,2045,142,252,373,2009-10
3,Joe Alexander,SF,23,CHI,8,0,3.6,0.1,0.8,0.167,0.0,0.1,0.000,0.1,0.6,0.200,0.167,0.3,0.4,0.667,0.3,0.4,0.6,0.3,0.1,0.1,0.0,1.1,0.5,0.167,9.3,2.6,-8.3,0.9,11.3,0.0,0.500,-9.1,7.8,0.0,2.8,1.8,0.0,9.6,0.273,11.3,0.0,0.0,0.030,104.0,88.0,0.0,0.0,0.0,0,0.0,0.0,0.0,-17.5,-18.9,14,0,4.0,86,0,5.0,0.0,669,1375,121,218,79,540,720,1496,536,586,3039,80,208,2027,70,149,620,2009-10
4,Malik Allen,PF,31,DEN,51,3,8.9,0.9,2.3,0.397,0.0,0.1,0.167,0.9,2.2,0.409,0.401,0.2,0.3,0.923,0.7,0.9,1.6,0.3,0.2,0.1,0.4,1.3,2.1,0.052,5.1,0.8,-5.7,-1.0,11.5,0.3,0.112,-4.7,9.2,-0.3,5.9,1.2,15.3,10.4,0.431,14.0,-0.4,0.1,0.009,110.0,93.0,1.0,8.0,4.0,50,7.0,4.0,2.0,-19.2,-12.9,50,0,40.0,0,0,37.0,6.0,617,1268,301,365,20,313,716,1569,439,541,2835,78,187,1970,113,233,391,2009-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,Dorell Wright,SF,24,MIA,72,1,20.8,2.7,5.8,0.463,0.8,2.2,0.389,1.8,3.6,0.508,0.536,0.8,1.0,0.884,0.7,2.6,3.3,1.3,0.7,0.4,0.7,1.3,7.1,0.375,9.8,1.7,1.8,1.2,14.4,2.2,0.165,0.7,4.0,2.1,14.5,1.9,10.6,9.2,0.567,15.8,1.5,4.3,0.137,104.0,114.0,7.0,31.0,10.0,0,19.0,0.0,4.0,-1.1,2.2,3,0,194.0,83,14,67.0,24.0,1,3,0,0,0,0,1,3,0,0,15,1,1,2,0,1,1,2009-10
438,Julian Wright,SF,22,NOH,68,14,12.8,1.7,3.4,0.500,0.0,0.1,0.333,1.7,3.3,0.504,0.504,0.4,0.6,0.610,0.9,1.3,2.1,0.6,0.4,0.3,0.6,0.7,3.8,0.026,7.0,1.6,-2.1,-0.5,11.8,0.6,0.177,-1.6,7.7,0.2,11.8,1.7,15.0,9.7,0.518,15.4,0.0,0.8,0.044,110.0,101.0,3.0,20.0,12.0,0,15.0,2.0,4.0,1.8,-1.5,34,0,87.0,63,4,31.0,17.0,1,1,0,0,0,1,1,1,0,0,5,0,0,2,1,0,1,2009-10
439,Nick Young,SG,24,WAS,74,23,19.2,3.1,7.5,0.418,0.9,2.3,0.406,2.2,5.2,0.423,0.479,1.4,1.8,0.800,0.3,1.1,1.4,0.6,0.4,0.1,0.8,2.0,8.6,0.305,5.7,0.4,-3.7,-1.7,6.7,0.5,0.233,-2.0,1.6,0.9,10.7,1.0,8.3,4.1,0.519,21.4,-0.6,1.4,0.046,113.0,104.0,7.0,19.0,20.0,0,17.0,8.0,15.0,3.9,-3.0,1,0,103.0,14,85,70.0,57.0,1,2,0,6,1,2,1,2,0,0,22,0,4,2,2,3,2,2009-10
440,Sam Young,SF,24,MEM,80,1,16.5,2.8,6.2,0.451,0.1,0.7,0.196,2.7,5.5,0.483,0.462,1.7,2.2,0.777,1.0,1.6,2.5,0.7,0.4,0.3,1.2,1.3,7.4,0.113,6.2,1.2,-3.1,-1.7,11.0,0.7,0.352,-1.5,6.8,0.6,13.0,1.3,14.1,8.9,0.518,22.1,-0.4,1.2,0.044,112.0,102.0,23.0,32.0,53.0,0,29.0,13.0,16.0,-9.1,-7.8,1,0,113.0,51,48,46.0,90.0,0,0,0,0,0,0

In [5]:

app = Dash() 

app.layout = html.Div([
    html.Div(children=[dcc.RadioItems(options=['PPG', 'APG', 'RPG', '3P%', 'FG%', 'FT%', 'STL', 'BLK', 'TOV', 'PF', 'G', 'MP', 'DRtg', 'ORtg', 'WS', 'DWS'], value='PPG', id='controls', inline=True)], style={'text-align':'center', 'font-family':'calibri'}), 
    html.Div(children=[dcc.Dropdown(master.Player.unique(), value='LeBron James', id='dropdown-selection')], style={'font-family':'calibri'}), 
    html.Div( children=[dcc.Graph(figure={}, id='graph-content')]) 
])

@callback(
    Output(component_id='graph-content', component_property='figure'),
    Input('controls', 'value'), Input('dropdown-selection', 'value')
)
def update_graph(stat, player): 
    player = master[master.Player==player] 
    fig = px.line(player, x='Season', y=stat, color_discrete_sequence=['orange'], markers=True)
    return fig 

if __name__ == '__main__':
    app.run(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on


SystemExit: 1

C:\Users\bchan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
